In [1]:
import pandas as pd
import numpy as np

In [2]:
stocks_df = pd.read_parquet('/Users/remibreton/Documents/data/stocks_data.parquet')

tickers = stocks_df.groupby('Ticker')['Close'].count().sort_values(ascending=False)
tickers = tickers[tickers == max(tickers)].index.to_list()
stocks_df = stocks_df[stocks_df['Ticker'].isin(tickers)]
close_df = stocks_df.pivot(index='Date', columns='Ticker', values='Close')

returns_df = close_df.pct_change()[1:]
returns_df = returns_df.fillna(0)
returns_df

Ticker,A,AA,AAME,AAON,AAP,AAPL,AB,ABCB,ABEO,ABEV,...,YHGJ,YORW,YPF,YUM,ZBH,ZBRA,ZD,ZEUS,ZION,ZTR
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-04,-0.026382,-0.018070,0.026059,-0.007329,-0.006659,0.010270,-0.025018,-0.017552,0.011628,0.000000,...,-0.098361,-0.006500,-0.003460,-0.013242,-0.000378,-0.018735,-0.034330,0.005948,-0.013652,-0.005650
2005-01-05,-0.000430,-0.005915,-0.015873,-0.003355,0.001849,0.008758,-0.003177,-0.031265,-0.043103,0.000000,...,0.115151,-0.044791,0.001852,-0.002381,-0.007814,-0.034882,-0.038308,-0.011431,-0.003611,-0.003788
2005-01-06,-0.021945,0.004298,-0.025806,-0.014815,-0.000923,0.000775,-0.000981,0.021004,0.000000,-0.020000,...,0.163043,-0.011591,0.007625,0.011282,0.011559,0.001522,-0.018802,-0.000797,0.005738,0.000000
2005-01-07,-0.000880,0.010204,0.062914,-0.012987,-0.005774,0.072811,-0.007853,-0.014049,-0.024024,0.002721,...,0.158878,-0.014392,-0.000229,-0.005578,-0.001758,0.008547,-0.012017,-0.003990,-0.024321,0.003802
2005-01-10,-0.004844,-0.007168,0.018692,-0.002078,0.003484,-0.004187,0.002473,0.018829,0.012308,0.000000,...,0.149194,0.001082,-0.000459,0.000216,0.009560,0.013936,0.009533,0.016426,0.000615,-0.003788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-05,0.036100,0.086664,0.031250,0.028918,-0.003600,-0.013837,0.016822,0.024061,-0.016949,0.016194,...,0.000000,-0.006938,0.002480,-0.001329,0.029216,0.017479,-0.005590,0.010030,0.017890,-0.009464
2026-01-06,0.032673,0.034505,0.006734,0.002332,0.067355,-0.018334,0.007636,0.005808,-0.011494,0.000000,...,0.011142,0.005081,-0.054700,0.007452,0.022666,0.019672,0.014201,0.011736,0.010778,0.004777
2026-01-07,0.001423,-0.012901,-0.003344,-0.049712,-0.003385,-0.007737,-0.015913,0.008793,0.011628,0.000000,...,0.011019,-0.002844,-0.046816,-0.006076,-0.021953,-0.041341,-0.013419,0.005130,-0.013287,-0.006339


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

class PortfolioDataset(Dataset):
    def __init__(self, returns_df: pd.DataFrame, lookback: int = 60, decision_step: int = 10, n_asset: int = 10, n_samples: int = 100_000):
        self.returns = returns_df.values.astype(np.float32)
        self.T, self.N = self.returns.shape

        self.lookback = lookback
        self.decision_step = decision_step
        self.window_length = lookback + decision_step

        self.k = n_asset
        self.n_samples = n_samples

    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        t = np.random.randint(self.window_length, self.T)
        asset_idx = np.random.choice(self.N, self.k, replace=False)
        window = self.returns[t - self.lookback:t, asset_idx]
        
        return {'input_r': torch.tensor(window), 'asset_idx': torch.tensor(asset_idx)}

num_timesteps, _ = returns_df.shape
train_lim = int(0.8 * num_timesteps)

train_df = returns_df[:train_lim]
test_df = returns_df[train_lim:]

train_dataset = PortfolioDataset(train_df)
test_dataset = PortfolioDataset(test_df)

train_dataloader = DataLoader(train_dataset, batch_size=256)
test_dataloader = DataLoader(test_dataset, batch_size=256)

In [4]:
import torch.nn.functional as F

class POptModel(torch.nn.Module):
    def __init__(self, n_asset: int, decision_step: int = 20, hidden_dim: int = 64, num_layers: int = 5):
        super().__init__()
        
        self.n_asset = n_asset
        self.decision_step = decision_step

        self.lstm = torch.nn.LSTM(
            input_size = n_asset,
            hidden_size = hidden_dim,
            num_layers = num_layers,
            batch_first = True
        )

        self.head = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, n_asset)
        )
    
    def forward(self, r):
        h_t, _ = self.lstm(r)
        #print('h_t.shape: ', h_t.shape)

        h_decision = h_t[: , self.decision_step:-1, :]
        #print('h_decision.shape: ', h_decision.shape)

        scores = self.head(h_decision)
        #print('scores.shape: ', scores.shape)

        w = F.softmax(scores, dim=-1)
        #print('w.shape: ', w.shape)

        next_r = r[:, self.decision_step+1:, :]
        #print('next_r.shape: ', next_r.shape)
        
        return w, next_r

In [ ]:
class SharpeLoss(torch.nn.Module):
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
    
    def forward(self, w, next_r):
        
        port_returns = torch.sum(w * next_r, dim=-1)
        #print('port_returns.shape: ', port_returns.shape)

        mean_t = port_returns.mean(dim=1) 
        var_t = port_returns.var(dim=1, unbiased=False)
        sharpe_t = mean_t / torch.sqrt(var_t + self.eps)

        return -sharpe_t.mean()

class WeightPenalty(torch.nn.Module):
    def __init__(self, param: float = 1.0):
        super().__init__()
        self.param = param
    
    def forward(self, w):
        delta_w = w[:, 1:, :] - w[:, :-1, :]
        penalty_k = torch.sum(torch.abs(delta_w), dim=1)
        penalty = torch.sum(penalty_k, dim=1)

        return self.param * penalty.mean()

In [22]:
from tqdm import tqdm

device = torch.device(
    "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f'using device: {device}')

model = POptModel(n_asset = train_dataset.k).to(device)

sharpe_crit = SharpeLoss().to(device)
weight_crit = WeightPenalty().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

n_epoch = 100
for epoch in range(n_epoch):
    model.train()
    train_loss = 0.0
    train_sharpe_loss = 0.0
    train_weight_loss = 0.0

    print(
        f"Running epoch {epoch+1:03d} ..."
    )
    
    for batch in tqdm(train_dataloader, desc="Train"):
        x = batch['input_r'].to(device)      # (B, L, K)

        optimizer.zero_grad()

        w, next_r = model(x)
        sharpe_loss = sharpe_crit(w, next_r)
        weight_loss = weight_crit(w)
        loss = sharpe_loss
        loss.backward()
        
        optimizer.step()

        train_loss += loss.item()
        train_sharpe_loss += sharpe_loss.item()
        train_weight_loss += weight_loss.item()

    train_loss /= len(train_dataloader)
    train_sharpe_loss /= len(train_dataloader)
    train_weight_loss /= len(train_dataloader)

    # --------------------
    # Evaluation
    # --------------------
    model.eval()
    test_loss = 0.0
    test_sharpe_loss = 0.0
    test_weight_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Eval"):
            x = batch['input_r'].to(device)

            w, next_r = model(x)
            sharpe_loss = sharpe_crit(w, next_r)
            weight_loss = weight_crit(w)
            loss = sharpe_loss + weight_loss

            test_loss += loss.item()
            test_sharpe_loss += sharpe_loss.item()
            test_weight_loss += weight_loss.item()

    test_loss /= len(test_dataloader)
    test_sharpe_loss /= len(test_dataloader)
    test_weight_loss /= len(test_dataloader)

    print(
        f"Epoch {epoch+1:03d} | "
        f"Train Loss       : {train_loss:.4f} | "
        f"Test Loss: {test_loss:.4f}               | \n"

        f"          | "
        f"Train Sharpe.    : {-train_sharpe_loss:.4f}  | "
        f"Test Sharpe: {-test_sharpe_loss:.4f}     | \n"

        f"          | "
        f"Train Weight Pen.: {train_weight_loss:.4f}  | "
        f"Test Weight Pen.: {test_weight_loss:.4f} | \n"
    )

using device: mps
Running epoch 001 ...


Eval: 100%|██████████| 391/391 [00:10<00:00, 35.98it/s]


Epoch 001 | Train Loss       : -0.0585 | Test Loss: -0.0236 | 
          | Train Sharpe.    : 0.0585  | Test Sharpe: 0.0236 | 
          | Train Weight Pen.: 0.0000  | Test Weight Pen.: 0.0000 | 

Running epoch 002 ...


Eval: 100%|██████████| 391/391 [00:09<00:00, 39.56it/s]


Epoch 002 | Train Loss       : -0.0592 | Test Loss: -0.0240 | 
          | Train Sharpe.    : 0.0592  | Test Sharpe: 0.0240 | 
          | Train Weight Pen.: 0.0000  | Test Weight Pen.: 0.0000 | 

Running epoch 003 ...


...

Epoch 026 | Train Loss       : -0.0595 | Test Loss: -0.0234 | 
          | Train Sharpe.    : 0.0595  | Test Sharpe: 0.0234 | 
          | Train Weight Pen.: 0.0000  | Test Weight Pen.: 0.0000 | 

Running epoch 027 ...


Eval: 100%|██████████| 391/391 [00:10<00:00, 37.15it/s]


Epoch 027 | Train Loss       : -0.0593 | Test Loss: -0.0232 | 
          | Train Sharpe.    : 0.0593  | Test Sharpe: 0.0232 | 
          | Train Weight Pen.: 0.0000  | Test Weight Pen.: 0.0000 | 

Running epoch 028 ...


Eval:  67%|██████▋   | 261/391 [00:07<00:03, 34.93it/s]


KeyboardInterrupt: 